In [9]:
import pandas as pd
import glob
from pathlib import Path

In [11]:
data_path = Path("../data")

files = [
    "PL_2018-2019.csv",
    "PL_2019-2020.csv",
    "PL_2020-2021.csv",
    "PL_2021-2022.csv",
    "PL_2022-2023.csv",
]

dfs = []
for f in files:
    df = pd.read_csv(data_path / f)
    df["season"] = f.replace(".csv", "")
    dfs.append(df)

In [12]:
dfs[0].head(), dfs[0].columns


(  Div        Date      HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
 0  E0  10/08/2018    Man United       Leicester     2     1   H     1     0   
 1  E0  11/08/2018   Bournemouth         Cardiff     2     0   H     1     0   
 2  E0  11/08/2018        Fulham  Crystal Palace     0     2   A     0     1   
 3  E0  11/08/2018  Huddersfield         Chelsea     0     3   A     0     2   
 4  E0  11/08/2018     Newcastle       Tottenham     1     2   A     1     2   
 
   HTR  ... BbAH  BbAHh  BbMxAHH  BbAvAHH  BbMxAHA  BbAvAHA  PSCH  PSCD  PSCA  \
 0   H  ...   17  -0.75     1.75     1.70     2.29     2.21  1.55  4.07  7.69   
 1   H  ...   20  -0.75     2.20     2.13     1.80     1.75  1.88  3.61  4.70   
 2   A  ...   22  -0.25     2.18     2.11     1.81     1.77  2.62  3.38  2.90   
 3   A  ...   23   1.00     1.84     1.80     2.13     2.06  7.24  3.95  1.58   
 4   A  ...   20   0.25     2.20     2.12     1.80     1.76  4.74  3.53  1.89   
 
          season  
 0  PL_2018

In [13]:
df = pd.concat(dfs, ignore_index=True)

df = df[[
    "Date",
    "HomeTeam",
    "AwayTeam",
    "FTR",
    "season"
]]

df.rename(columns={
    "Date": "date",
    "HomeTeam": "home_team",
    "AwayTeam": "away_team",
    "FTR": "result"
}, inplace=True)


In [14]:
df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors="coerce")
df.isna().sum()


date         0
home_team    0
away_team    0
result       0
season       0
dtype: int64

In [15]:
mapping = {"H": 0, "D": 1, "A": 2}
df["result_code"] = df["result"].map(mapping)

In [16]:
df["result_code"].value_counts()

result_code
0    844
2    635
1    421
Name: count, dtype: int64

In [17]:
df = df.sort_values("date").reset_index(drop=True)

In [18]:
print("Number of matches:", len(df))
n_teams = pd.concat([df["home_team"], df["away_team"]]).nunique()
print("Number of teams:", n_teams)
print(df.head())

Number of matches: 1900
Number of teams: 27
        date     home_team       away_team result        season  result_code
0 2018-08-10    Man United       Leicester      H  PL_2018-2019            0
1 2018-08-11   Bournemouth         Cardiff      H  PL_2018-2019            0
2 2018-08-11        Fulham  Crystal Palace      A  PL_2018-2019            2
3 2018-08-11  Huddersfield         Chelsea      A  PL_2018-2019            2
4 2018-08-11     Newcastle       Tottenham      A  PL_2018-2019            2


In [19]:
output_path = Path("../cleaned_football_data.csv")
df.to_csv(output_path, index=False)